When reviewing SPM tissue segmentation QC results, I found the occurence of GM underesttimation to be more common than previously thought. I create a new pipeline to compute mean and SD in each tissue compartment for both SPM and Freesufer, so that we can quantify the quality of segmentation in the two, and also find any outliers for T1/T2flair intensity levels in different compartments.

In this notebook I will merge individual csvs generated by the pipeline and make interactive plots for comparing algos.

Also, Antonietta has computed FS6 segmentation QC metric by computing the discrepancy between atlas labels and segmented labels in each subject, roughly aligned in the native space. The analysis and summary of these metrics will be performed in the PART II section of the notebook.

In [1]:
import os
import os.path as op
import numpy as np
import pandas as pd
import glob

In [2]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

In [15]:
from plotting_tools.interactive_plots import plot_hist_box, pairplots_by_region
from plotting_tools.utils import get_bokehpalette
from bokeh.layouts import gridplot, row, column
from bokeh.models import Div, Spacer
import bokeh.io

/homes_unix/tsuchida/anaconda2/envs/py3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [18]:
# Get subject list for included subjects

included_df = pd.read_csv("MRiShare_dat/MRiShare_included_demographics_20190320.csv")
included_df.head()

,SHARE_id,Sex,Handedness,DOB,DOA,Age
0,SHARE0001,M,D,07/08/1991,03/11/2015,"24,241"
1,SHARE0002,F,D,24/11/1992,04/11/2015,"22,943"
2,SHARE0003,F,D,17/03/1992,04/11/2015,"23,633"
3,SHARE0004,M,D,24/09/1992,03/11/2015,"23,107"
4,SHARE0005,F,D,26/10/1990,03/11/2015,"25,021"


In [19]:
included_subs = included_df['SHARE_id'].values

In [68]:
len(included_subs)

1834

# PART I. Struc suppl QC: SNR and CNR

## 1) data dirs

In [51]:
supplQCsink = "/beegfs_data/scratch/tsuchida-anatQC/StructQCSink/"

In [52]:
out_dir = "MRiShare_struct_supplQC_Mar2019_out"
os.makedirs(out_dir, exist_ok=True)

## 2) Merge CSVs

In [53]:
from waimea_tools.interfaces.custom import merge_csvs

In [54]:
algos = ['spm', 'fsaseg']
stats = ['T1', 'T2flair']

In [75]:
df_dict = {}

for algo in algos:
    weights = [''] if algo == 'fsaseg' else ['weighted_', 'unweighted_']
    for weight in weights:
        df_dict['{}{}'.format(weight, algo)] = {}
        for stat in stats:
            stat_dir = op.join(supplQCsink, '{}{}stats_{}'.format(weight, stat, algo))
            subdir_glob = glob.glob(op.join(stat_dir, '_subject_id*'))
            sub_ids = [op.basename(subdir).replace('_subject_id_', '') for subdir in sorted(subdir_glob)]
            csv_glob = glob.glob(op.join(stat_dir, '_subject_id*', '*.csv'))
            out_csvname = op.join(out_dir, '{}{}stats_{}_merged_{}.csv'.format(weight, stat, algo, len(csv_glob)))
            out_csv = merge_csvs(sorted(csv_glob), out_csvname,
                                new_idx_name="SHARE_id", new_idx_vals=sub_ids)
            df_dict['{}{}'.format(weight, algo)][stat] = pd.read_csv(out_csv, index_col=0)

In [76]:
len(df_dict['fsaseg']['T1'])

1873

## 3) Add SNR and CNR

In each DF, add the following;

* 1) Tissue SNR
   * computed as mean/sd in each tissue in each compartments
* 2) Tissue CNR
    * for T1 stats, WMGM (WM mean/GM mean)and GMCSF (GM mean/CSF mean)
    * for T2flair stats, GMWM (GM mean/ WM mean) and GMCSF (GM mean/CSF mean)
                 

In [57]:
tissue_types = ['GM', 'WM', 'CSF']
compartments = ['hemiR', 'hemiL', 'cerebellarR', 'cerebellarL']

In [77]:
for key, d in df_dict.items():
    for stat_key, df in d.items():
        # First add SNR
        mean_cols = [col for col in df.columns if 'mean' in col]
        sd_cols = [col for col in df.columns if 'sd' in col]
        for mean_col, sd_col in zip(mean_cols, sd_cols):
            df[mean_col.replace('mean', 'SNR')] = df[mean_col] / df[sd_col]
            
        # Next add CNR
        for c in compartments:
            c_cols = [col for col in df.columns if c in col and 'mean' in col]
            tissue_col_d = {tissue: [col for col in c_cols if tissue in col][0] for tissue in tissue_types}
            
            if stat_key == 'T1':
                df['{}_WMGM_CNR'.format(c)] = df[tissue_col_d['WM']] /  df[tissue_col_d['GM']]
            else:
                df['{}_GMWM_CNR'.format(c)] = df[tissue_col_d['GM']] /  df[tissue_col_d['WM']]
                
            df['{}_GMCSF_CNR'.format(c)] = df[tissue_col_d['GM']] /  df[tissue_col_d['CSF']]

In [59]:
c_cols

['GM_cerebellarL_mean', 'WM_cerebellarL_mean', 'CSF_cerebellarL_mean']

In [60]:
[col for col in c_cols if 'GM' in col]

['GM_cerebellarL_mean']

In [78]:
df_dict['fsaseg']['T2flair'][[col for col in df_dict['fsaseg']['T2flair'].columns if 'hemiR' in col]].head()

,GM_hemiR_mean,GM_hemiR_sd,GM_hemiR_size,WM_hemiR_mean,WM_hemiR_sd,WM_hemiR_size,CSF_hemiR_mean,CSF_hemiR_sd,CSF_hemiR_size,GM_hemiR_SNR,WM_hemiR_SNR,CSF_hemiR_SNR,hemiR_GMWM_CNR,hemiR_GMCSF_CNR
SHARE_id,,,,,,,,,,,,,,
SHARE0001,185.204605,22.520420,1049895,150.854874,14.844149,843651,60.051758,50.128937,34122,8.223852,10.162582,1.197946,1.227701,3.084083
SHARE0002,202.633789,23.687967,852393,159.636902,16.532265,625356,108.575577,55.163044,11844,8.554292,9.656082,1.968267,1.269342,1.866293
SHARE0003,199.154602,20.677555,849645,161.678452,15.962511,608688,139.359665,60.264500,8103,9.631439,10.128635,2.312467,1.231794,1.429069
SHARE0004,178.085388,21.898132,1033635,142.819061,14.609125,885594,59.611935,42.247707,34416,8.132446,9.776017,1.411010,1.246930,2.987412
SHARE0005,202.065048,24.500608,714162,159.414764,17.595995,577977,122.885193,61.259869,10581,8.247348,9.059719,2.005966,1.267543,1.644340


In [62]:
df_dict['unweighted_spm']['T2flair'][[col for col in df_dict['unweighted_spm']['T2flair'].columns if 'hemiR' in col]].head()

,GM_hemiR_mean,GM_hemiR_sd,GM_hemiR_size,WM_hemiR_mean,WM_hemiR_sd,WM_hemiR_size,CSF_hemiR_mean,CSF_hemiR_sd,CSF_hemiR_size,GM_hemiR_SNR,WM_hemiR_SNR,CSF_hemiR_SNR,hemiR_GMWM_CNR,hemiR_GMCSF_CNR
SHARE_id,,,,,,,,,,,,,,
SHARE0001,188.281281,16.150103,903090,145.557755,10.328842,686268,141.189026,51.301159,521679,11.658210,14.092359,2.752161,1.293516,1.333540
SHARE0002,204.929337,18.648647,743061,153.519165,11.096687,515376,161.238480,48.875408,383049,10.988965,13.834684,3.298969,1.334878,1.270970
SHARE0003,201.917343,15.742546,743349,156.409241,11.246231,509250,169.233978,39.361279,311373,12.826219,13.907703,4.299504,1.290955,1.193125
SHARE0004,180.371796,16.781809,900378,137.002579,9.401596,704583,134.350113,48.038811,559284,10.748054,14.572268,2.796699,1.316558,1.342550
SHARE0005,203.447205,19.523302,624456,151.890594,10.798853,449268,168.112183,46.195564,322251,10.420737,14.065438,3.639141,1.339433,1.210187


In [63]:
df_dict['weighted_spm']['T2flair'][[col for col in df_dict['weighted_spm']['T2flair'].columns if 'hemiR' in col]].head()

,GM_hemiR_weighted_mean,GM_hemiR_weighted_sd,GM_hemiR_weighted_size,WM_hemiR_weighted_mean,WM_hemiR_weighted_sd,WM_hemiR_weighted_size,CSF_hemiR_weighted_mean,CSF_hemiR_weighted_sd,CSF_hemiR_weighted_size,GM_hemiR_weighted_SNR,WM_hemiR_weighted_SNR,CSF_hemiR_weighted_SNR,hemiR_GMWM_CNR,hemiR_GMCSF_CNR
SHARE_id,,,,,,,,,,,,,,
SHARE0001,187.711814,16.391534,294209.323282,145.635195,10.469483,227774.154647,143.400654,52.211095,181823.022518,11.451754,13.910447,2.746555,1.288918,1.309002
SHARE0002,204.324851,18.876165,242470.414339,153.608871,11.276097,171145.798357,163.636041,50.025485,133065.066693,10.824490,13.622521,3.271054,1.330163,1.248654
SHARE0003,201.332979,15.918990,240668.288743,156.496625,11.419986,169261.343343,172.070167,40.526723,111026.383037,12.647346,13.703749,4.245845,1.286500,1.170063
SHARE0004,179.797624,17.000929,294533.778203,137.046070,9.512859,233540.896164,136.280066,48.957259,193085.744752,10.575753,14.406401,2.783654,1.311950,1.319324
SHARE0005,202.686636,19.734742,203198.055154,151.960616,10.971811,149026.981362,170.564575,47.123947,112725.908627,10.270549,13.850094,3.619488,1.333810,1.188328


In [79]:
# Merge across T1 or T2flair stats and save

merged_df_dict = {}
for stat in stats:
    df_list = []
    for key, d in df_dict.items():
        df = d[stat].copy()
        df.columns = ['{}_{}'.format(key, col) for col in df.columns]
        df_list.append(df)
        
    merged_df = pd.concat(df_list, axis=1)
    merged_df.to_csv(op.join(out_dir, 'FS6aseg_SPM_{}stats_{}.csv'.format(stat, len(merged_df))))
    merged_df_dict[stat] = merged_df

In [80]:
df_list = []
for key, d in df_dict.items():
    for stat_key, df in d.items():
        prefix = stat_key.replace('weighted_', '').replace('unweighted_', '')
        df.columns = ['{}_{}_{}'.format(key, stat_key, col) for col in df.columns]
        df_list.append(df)
        
all_merged_df = pd.concat(df_list, axis=1)
all_merged_df.to_csv(op.join(out_dir, 'mergedT1T2stats_{}.csv'.format(len(all_merged_df))))

## 4) Comparisons of FSaseg vs SPM
Since weighted and unweighted SPM are not so different, compare FSaseg against weighted SPM values.

In [66]:
fs_spm_out = op.join(out_dir, 'FS_SPM_comparison_plots')
os.makedirs(fs_spm_out, exist_ok=True)

In [81]:
# Plots of tissue mean T1/T2flair intensity (only included subjects)
my_palette = get_bokehpalette('Spectral', len(tissue_types))
for stat in stats:
    tissue_col = []
    for i, tissue in enumerate(tissue_types):
        title = "<h3>Mean {} values in {} by FSaseg vs SPM comparison</h3>".format(stat, tissue)
        pplots = []
        for mask in compartments:
            col1 = "fsaseg_{}_{}_mean".format(tissue, mask)
            col2 = "weighted_spm_{}_{}_weighted_mean".format(tissue, mask)
            pplot = pairplots_by_region(merged_df_dict[stat].loc[included_subs],
                                        stat,
                                        col1=col1, col2=col2,
                                        plot_size=(400, 400),
                                        title='Mean {} in {} {}'.format(stat, mask, tissue),
                                        color=my_palette[i])
            pplots.append(pplot)
            
        tissue_col.append(column([Div(text=title),
                                  gridplot(pplots, ncols=2)],
                                 sizing_mode='fixed'))
                          
        if i+1 < (len(tissue_types)):
            tissue_col.append(Spacer(width=100))
        
    final_plot = row(tissue_col)
    
    bokeh.io.save(final_plot,
                  op.join(fs_spm_out,
                          'Tissue_{}_values_FS_vs_SPM_{}.html'.format(stat, len(included_subs))),
                  title='Comparison of regional {} values with FS and SPM segmentations'.format(stat))
            
        

/homes_unix/tsuchida/anaconda2/envs/py3/lib/python3.6/site-packages/bokeh/io/saving.py:125: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/homes_unix/tsuchida/anaconda2/envs/py3/lib/python3.6/site-packages/bokeh/io/saving.py:125: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")


In [82]:
# Plots of tissue SNR
for stat in stats:
    tissue_col = []
    for i, tissue in enumerate(tissue_types):
        title = "<h3>{} SNR values in {} by FSaseg vs SPM comparison</h3>".format(stat, tissue)
        pplots = []
        for mask in compartments:
            col1 = "fsaseg_{}_{}_SNR".format(tissue, mask)
            col2 = "weighted_spm_{}_{}_weighted_SNR".format(tissue, mask)
            pplot = pairplots_by_region(merged_df_dict[stat].loc[included_subs],
                                        'SNR',
                                        col1=col1, col2=col2,
                                        plot_size=(400, 400),
                                        title='{} SNR in {} {}'.format(stat, mask, tissue),
                                        color=my_palette[i])
            pplots.append(pplot)
            
        tissue_col.append(column([Div(text=title),
                                  gridplot(pplots, ncols=2)],
                                 sizing_mode='fixed'))
                          
        if i+1 < (len(tissue_types)):
            tissue_col.append(Spacer(width=100))
        
    final_plot = row(tissue_col)
    
    bokeh.io.save(final_plot,
                  op.join(fs_spm_out,
                          'Tissue_{}_SNR_FS_vs_SPM_{}.html'.format(stat, len(included_subs))),
                  title='Comparison of regional {} SNR with FS and SPM segmentations'.format(stat))

/homes_unix/tsuchida/anaconda2/envs/py3/lib/python3.6/site-packages/bokeh/io/saving.py:125: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/homes_unix/tsuchida/anaconda2/envs/py3/lib/python3.6/site-packages/bokeh/io/saving.py:125: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")


In [83]:
# Plots of tissue CNR
CNR_palette = get_bokehpalette("Spectral", 5)

for stat in stats:
    cnr_col = []
    cnr_list = ['WMGM', 'GMCSF'] if stat == 'T1' else ['GMWM', 'GMCSF']
    for i, cnr in enumerate(cnr_list):
        title = "<h3>{} {} CNR values by FSaseg vs SPM comparison</h3>".format(stat, cnr)
        pplots = []
        for mask in compartments:
            col1 = "fsaseg_{}_{}_CNR".format(mask, cnr)
            col2 = "weighted_spm_{}_{}_CNR".format(mask, cnr)
            pplot = pairplots_by_region(merged_df_dict[stat].loc[included_subs],
                                        'CNR',
                                        col1=col1, col2=col2,
                                        plot_size=(400, 400),
                                        title='{} {} CNR in {}'.format(stat, cnr, mask),
                                        color=CNR_palette[(i*2)+1])
            pplots.append(pplot)
            
        cnr_col.append(column([Div(text=title),
                               gridplot(pplots, ncols=2)],
                               sizing_mode='fixed'))
                          
        if i+1 < (len(cnr_list)):
            cnr_col.append(Spacer(width=100))
        
    final_plot = row(cnr_col)
    
    bokeh.io.save(final_plot,
                  op.join(fs_spm_out,
                          '{}_CNR_FS_vs_SPM_{}.html'.format(stat, len(included_subs))),
                  title='Comparison of {} tissue CNR with FS and SPM segmentations'.format(stat))

/homes_unix/tsuchida/anaconda2/envs/py3/lib/python3.6/site-packages/bokeh/io/saving.py:125: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/homes_unix/tsuchida/anaconda2/envs/py3/lib/python3.6/site-packages/bokeh/io/saving.py:125: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")


In [83]:
merged_df_dict['T1'].columns

Index(['weighted_spm_GM_hemiR_weighted_mean',
       'weighted_spm_GM_hemiR_weighted_sd',
       'weighted_spm_GM_hemiR_weighted_size',
       'weighted_spm_GM_hemiL_weighted_mean',
       'weighted_spm_GM_hemiL_weighted_sd',
       'weighted_spm_GM_hemiL_weighted_size',
       'weighted_spm_GM_cerebellarR_weighted_mean',
       'weighted_spm_GM_cerebellarR_weighted_sd',
       'weighted_spm_GM_cerebellarR_weighted_size',
       'weighted_spm_GM_cerebellarL_weighted_mean',
       ...
       'fsaseg_CSF_cerebellarR_SNR', 'fsaseg_CSF_cerebellarL_SNR',
       'fsaseg_hemiR_WMGM_CNR', 'fsaseg_hemiR_GMCSF_CNR',
       'fsaseg_hemiL_WMGM_CNR', 'fsaseg_hemiL_GMCSF_CNR',
       'fsaseg_cerebellarR_WMGM_CNR', 'fsaseg_cerebellarR_GMCSF_CNR',
       'fsaseg_cerebellarL_WMGM_CNR', 'fsaseg_cerebellarL_GMCSF_CNR'],
      dtype='object', length=168)

## 5) R vs L asymmetry for QC

Again using weighted SPM and fsaseg, make sure there are no unknown outliers for asymmetry in;

* Tissue mean values
* Tissue SNR values
* Tissue CNR values

In [72]:
asym_out = op.join(out_dir, 'T1T2stats_asymmetry_plots')
os.makedirs(asym_out, exist_ok=True)

In [73]:
hemis = ['R', 'L']
regions = ['hemi', 'cerebellar']

In [84]:
# Tissue mean values
for stat in stats:
    for key in df_dict.keys():
        fname = 'Tissue_{}stat_asymmetry_{}_{}.html'.format(stat, key, len(included_subs))
        ftitle = 'Asymmetry of mean {} values in each tissue with {} segmentation'.format(stat, key)
        tissue_row = []
        for i, tissue in enumerate(tissue_types):
            title = "<h3>Mean {} value asymmetry in {} by {}</h3>".format(stat, tissue, key)
            pplots = []
            for region in regions:
                weight = "weighted_" if key == "weighted_spm" else ""
                col1 = "{}_{}_{}R_{}mean".format(key, tissue, region, weight)
                col2 = "{}_{}_{}L_{}mean".format(key, tissue, region, weight)
                pplot = pairplots_by_region(merged_df_dict[stat].loc[included_subs],
                                            stat,
                                            col1=col1, col2=col2,
                                            plot_size=(400, 400),
                                            title='Mean {} asymmetry in {} {}'.format(stat, region, tissue),
                                            color=my_palette[i])
                pplots.append(pplot)
            
            tissue_row.append(column([Div(text=title),
                                      row(pplots)],
                                     sizing_mode='fixed'))
                          
            if i+1 < (len(tissue_types)):
                tissue_row.append(Spacer(height=50))
        
        final_plot = column(tissue_row)
    
        bokeh.io.save(final_plot,
                      op.join(asym_out, fname),
                      title=ftitle)

/homes_unix/tsuchida/anaconda2/envs/py3/lib/python3.6/site-packages/bokeh/io/saving.py:125: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/homes_unix/tsuchida/anaconda2/envs/py3/lib/python3.6/site-packages/bokeh/io/saving.py:125: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/homes_unix/tsuchida/anaconda2/envs/py3/lib/python3.6/site-packages/bokeh/io/saving.py:125: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to

In [86]:
# Tissue SNR values
for stat in stats:
    for key in df_dict.keys():
        fname = 'Tissue_{}_SNR_asymmetry_{}_{}.html'.format(stat, key, len(included_subs))
        ftitle = 'Asymmetry of {} SNR values in each tissue with {} segmentation'.format(stat, key)
        tissue_row = []
        for i, tissue in enumerate(tissue_types):
            title = "<h3>{} SNR value asymmetry in {} by {}</h3>".format(stat, tissue, key)
            pplots = []
            for region in regions:
                weight = "weighted_" if key == "weighted_spm" else ""
                col1 = "{}_{}_{}R_{}SNR".format(key, tissue, region, weight)
                col2 = "{}_{}_{}L_{}SNR".format(key, tissue, region, weight)
                pplot = pairplots_by_region(merged_df_dict[stat].loc[included_subs],
                                            'SNR',
                                            col1=col1, col2=col2,
                                            plot_size=(400, 400),
                                            title='{} SNR asymmetry in {} {}'.format(stat, region, tissue),
                                            color=my_palette[i])
                pplots.append(pplot)
            
            tissue_row.append(column([Div(text=title),
                                      row(pplots)],
                                     sizing_mode='fixed'))
                          
            if i+1 < (len(tissue_types)):
                tissue_row.append(Spacer(height=50))
        
        final_plot = column(tissue_row)
    
        bokeh.io.save(final_plot,
                      op.join(asym_out, fname),
                      title=ftitle)

/homes_unix/tsuchida/anaconda2/envs/py3/lib/python3.6/site-packages/bokeh/io/saving.py:125: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/homes_unix/tsuchida/anaconda2/envs/py3/lib/python3.6/site-packages/bokeh/io/saving.py:125: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/homes_unix/tsuchida/anaconda2/envs/py3/lib/python3.6/site-packages/bokeh/io/saving.py:125: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to

In [87]:
# Tissue CNR values
for stat in stats:
    for key in df_dict.keys():
        fname = 'Tissue_{}_CNR_asymmetry_{}_{}.html'.format(stat, key, len(included_subs))
        ftitle = 'Asymmetry of {} tissue CNR values with {} segmentation'.format(stat, key)
        cnr_row = []
        cnr_list = ['WMGM', 'GMCSF'] if stat == 'T1' else ['GMWM', 'GMCSF']
        for i, cnr in enumerate(cnr_list):
            title = "<h3>{} {} CNR value asymmetry by {}</h3>".format(stat, cnr, key)
            pplots = []
            for region in regions:
                col1 = "{}_{}R_{}_CNR".format(key, region, cnr)
                col2 = "{}_{}L_{}_CNR".format(key, region, cnr)
                pplot = pairplots_by_region(merged_df_dict[stat].loc[included_subs],
                                            'CNR',
                                            col1=col1, col2=col2,
                                            plot_size=(400, 400),
                                            title='{} {} CNR asymmetry in {}'.format(stat, cnr, region),
                                            color=CNR_palette[(i*2)+1])
                pplots.append(pplot)
            
            cnr_row.append(column([Div(text=title),
                                   row(pplots)],
                                   sizing_mode='fixed'))
                          
            if i+1 < (len(cnr_list)):
                cnr_row.append(Spacer(height=50))
        
        final_plot = column(cnr_row)
    
        bokeh.io.save(final_plot,
                      op.join(asym_out, fname),
                      title=ftitle)

/homes_unix/tsuchida/anaconda2/envs/py3/lib/python3.6/site-packages/bokeh/io/saving.py:125: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/homes_unix/tsuchida/anaconda2/envs/py3/lib/python3.6/site-packages/bokeh/io/saving.py:125: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/homes_unix/tsuchida/anaconda2/envs/py3/lib/python3.6/site-packages/bokeh/io/saving.py:125: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to

# PART II. Freesurfer segmentation QC

## 1) data dir

There are following files:

* Volume discrepancy
    * overall
    * mean
    * std
    * for 12 structures
* Label discrepancy ( for R and L separately)
    * aparc
    * aparc.a2009s

In [6]:
fs_QC_dir = '/data/analyses/work_in_progress/freesurfer/QC_fsmrishare-flair6.0/DICE_coeff/'

In [16]:
fs_qc_out = op.join(out_dir, 'FS_DICE_QC')
os.makedirs(fs_qc_out, exist_ok=True)

## 2) Summarize Volume discrepancy

In [7]:
vol_dice_types = ['overall', 'mean', 'std', '12structures']

In [25]:
vol_dice_df_list = []
for vol_dice in vol_dice_types:
    df = pd.read_csv(op.join(fs_QC_dir, 'summary_vol_DICE_{}_MNI152.txt'.format(vol_dice)), index_col=0)
    vol_dice_df_list.append(df)
    
vol_dice_df = pd.concat(vol_dice_df_list, axis=1)

In [26]:
vol_dice_df.head()

,DICE_overall_MNI152,DICE_mean_MNI152,DICE_std_MNI152,dice_L_Hippocampus,dice_R_Hippocampus,dice_L_Caudate,dice_R_Caudate,dice_L_Putamen,dice_R_Putamen,dice_L_Pallidum,...,dice_R_Inf_Lat_Vent,dice_L_Cerebral_White_Matter,dice_R_Cerebral_White_Matter,dice_L_CerebralCortex,dice_R_CerebralCortex,dice_L_Accumbens_area,dice_R_Accumbens_area,mean,std,overall
subject,,,,,,,,,,,,,,,,,,,,,
SHARE0001,0.7023,0.6405,0.1831,0.6828,0.6953,0.7191,0.7246,0.6309,0.6233,0.6373,...,0.8170,0.3818,0.4704,0.7419,0.5750,0.1432,0.2093,0.6405,0.1831,0.7023
SHARE0002,0.5542,0.5528,0.1937,0.6750,0.6838,0.7368,0.7403,0.3597,0.3607,0.6734,...,0.7959,0.3899,0.5622,0.5004,0.5967,0.0313,0.2009,0.5528,0.1937,0.5542
SHARE0003,0.5798,0.5862,0.1917,0.6741,0.6878,0.7391,0.7421,0.2887,0.2637,0.7342,...,0.8059,0.2883,0.5515,0.5609,0.3594,0.3039,0.3841,0.5862,0.1917,0.5798
SHARE0004,0.6926,0.6501,0.1586,0.7036,0.7134,0.7223,0.7219,0.5558,0.6577,0.7403,...,0.7633,0.4888,0.5762,0.6686,0.5466,0.2243,0.2425,0.6501,0.1586,0.6926
SHARE0005,0.5933,0.5816,0.1892,0.6850,0.6861,0.7261,0.7214,0.3841,0.3742,0.8155,...,0.6378,0.4441,0.4647,0.5710,0.7956,0.1926,0.2135,0.5816,0.1892,0.5933


In [13]:
structures = [col.replace('dice_R_', '') for col in vol_dice_df.columns if 'dice_R' in col]

In [27]:
# Make a summary plot 

my_palette = get_bokehpalette('Spectral', len(structures) + 2)
dist_cols = ['DICE_{}_MNI152'.format(dice_type) for dice_type in vol_dice_types[:-1]]
dist_plot = plot_hist_box(vol_dice_df,
                          'volume_DICE',
                          'DICEtypes',
                          dist_cols[:-1],
                          palette=my_palette,
                          title='Distribution of volume DICE coefficients')

global_pplots = []
for i, dice_type1 in enumerate(vol_dice_types[:2]):
    col1 = dist_cols[i]
    col2 = dist_cols[i+1]
    dice_type2 = vol_dice_types[i+1]
    pplot = pairplots_by_region(vol_dice_df,
                                'volume_DICE',
                                col1=col1, col2=col2,
                                plot_size=(400, 400),
                                color=my_palette[i],
                                title='Comparison of {} and {} volume DICE'.format(dice_type1, dice_type2))
    global_pplots.append(pplot)
    
    
region_pplots = []
for i, structure in enumerate(structures):
    col1 = 'dice_L_{}'.format(structure)
    col2 = 'dice_R_{}'.format(structure)
    pplot = pairplots_by_region(vol_dice_df,
                                'volume_DICE',
                                col1=col1, col2=col2,
                                plot_size=(400, 400),
                                color=my_palette[i+2],
                                title='Asymmetry of volume DICE in {}'.format(structure))
    region_pplots.append(pplot)
    
final_plot = row([column([dist_plot, Spacer(height=50), row(global_pplots, sizing_mode='fixed')]),
                  Spacer(width=100),
                  gridplot(region_pplots, ncols=3)])

bokeh.io.save(final_plot, op.join(fs_qc_out, 'FS6_Volume_DICE_summary.html'), title='Summary of volume DICE')


/homes_unix/tsuchida/anaconda2/envs/py3/lib/python3.6/site-packages/bokeh/io/saving.py:125: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")


'/homes_unix/tsuchida/MRiSHARE/MRiShare_struct_supplQC_Mar2019_out/FS_DICE_QC/FS6_Volume_DICE_summary.html'

In [20]:
len(included_subs)

1834

In [43]:
# Repeat above with included subjects
df_to_plot = vol_dice_df.loc[included_subs, :]
dist_plot = plot_hist_box(df_to_plot,
                          'volume_DICE',
                          'DICEtypes',
                          dist_cols[:-1],
                          palette=my_palette,
                          title='Distribution of volume DICE coefficients')

global_pplots = []
for i, dice_type1 in enumerate(vol_dice_types[:2]):
    col1 = dist_cols[i]
    col2 = dist_cols[i+1]
    dice_type2 = vol_dice_types[i+1]
    pplot = pairplots_by_region(df_to_plot,
                                'volume_DICE',
                                col1=col1, col2=col2,
                                plot_size=(600, 600),
                                color=my_palette[i],
                                title='Comparison of {} and {} volume DICE'.format(dice_type1, dice_type2))
    global_pplots.append(pplot)
    
    
region_pplots = []
for i, structure in enumerate(structures):
    col1 = 'dice_L_{}'.format(structure)
    col2 = 'dice_R_{}'.format(structure)
    pplot = pairplots_by_region(df_to_plot,
                                'volume_DICE',
                                col1=col1, col2=col2,
                                plot_size=(400, 400),
                                color=my_palette[i+2],
                                title='Asymmetry of volume DICE in {}'.format(structure))
    region_pplots.append(pplot)
    
final_plot = row([column([dist_plot, Spacer(height=50), global_pplots[0], global_pplots[1]]),
                  Spacer(width=100),
                  gridplot(region_pplots, ncols=3)])

bokeh.io.save(final_plot, op.join(fs_qc_out, 'FS6_Volume_DICE_summary_{}.html'.format(len(df_to_plot))), title='Summary of volume DICE')

/homes_unix/tsuchida/anaconda2/envs/py3/lib/python3.6/site-packages/bokeh/io/saving.py:125: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")


'/homes_unix/tsuchida/MRiSHARE/MRiShare_struct_supplQC_Mar2019_out/FS_DICE_QC/FS6_Volume_DICE_summary_1834.html'

# 3) Summarize surface parcellation discrepancy

In [38]:
surfs = ['rh', 'lh']
parcellations = {'Desikan':'aparc', 'Destrieux': 'aparc.a2009s'}

In [39]:
surf_dice_df_list = []

for atlas, parc in parcellations.items():
    for surf in surfs:
        csv = op.join(fs_QC_dir, 'summary_DICE_surf_{}_{}_native_space.txt'.format(surf, parc))
        df = pd.read_csv(csv, index_col=0)
        df.columns = ['{}_{}_{}'.format(atlas, surf, col) for col in df.columns]
        surf_dice_df_list.append(df)
    
surf_dice_df = pd.concat(surf_dice_df_list, axis=1)

In [47]:
surf_dice_df.tail()

,Desikan_rh_dice,Desikan_rh_displacement,Desikan_lh_dice,Desikan_lh_displacement,Destrieux_rh_dice,Destrieux_rh_displacement,Destrieux_lh_dice,Destrieux_lh_displacement
subject,,,,,,,,
SHARE1996,0.943293,1.325353,0.940365,1.476231,0.827501,2.019629,0.844203,1.895427
SHARE1997,0.942671,1.284496,0.941013,1.236692,0.832705,1.818878,0.840546,1.785886
SHARE1998,0.938775,1.251969,0.938141,1.216542,0.831056,1.773998,0.848972,1.641334
SHARE1999,0.942503,1.374581,0.939698,1.327685,0.843153,1.806991,0.845724,1.786732
SHARE2000,0.944000,1.442880,0.937235,1.739190,0.831212,2.053676,0.843468,2.015214


In [41]:
measures = ['dice', 'displacement']

In [49]:
# Make sumamry of distribution and asymmetry of dice and displacemnt values

for measure in measures:
    cols_to_plot = [col for col in surf_dice_df if measure in col]
    my_palette = get_bokehpalette('Spectral', len(cols_to_plot))
    dist_plot = plot_hist_box(surf_dice_df,
                              'surf_{}'.format(measure),
                              'category',
                              cols_to_plot,
                              title='Distribution of surface {} values'.format(measure))
    
    pplots = []
    for i, (atlas, parc) in enumerate(parcellations.items()):
        col1 = '{}_lh_{}'.format(atlas, measure)
        col2 = '{}_rh_{}'.format(atlas, measure)
        pplot = pairplots_by_region(surf_dice_df,
                                    'surf_{}'.format(measure),
                                    col1=col1, col2=col2,
                                    plot_size=(600, 600),
                                    color=my_palette[i*2],
                                    title='Asymmetry of surface {} in {} atlas'.format(measure, atlas))
        pplots.append(pplot)
        
    final_plot = row([dist_plot,
                      Spacer(width=100),
                      column(pplots)])
    
    bokeh.io.save(final_plot, op.join(fs_qc_out, 'FS6_Surface_{}_summary.html'.format(measure)),
                  title='Summary of distributions of surface {}'.format(measure))

/homes_unix/tsuchida/anaconda2/envs/py3/lib/python3.6/site-packages/bokeh/io/saving.py:125: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/homes_unix/tsuchida/anaconda2/envs/py3/lib/python3.6/site-packages/bokeh/io/saving.py:125: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")


In [50]:
surf_dice_df.loc['SHARE1080']

Desikan_rh_dice              0.940858
Desikan_rh_displacement      1.220737
Desikan_lh_dice              0.936387
Desikan_lh_displacement      1.314422
Destrieux_rh_dice            0.827755
Destrieux_rh_displacement    1.829510
Destrieux_lh_dice            0.841684
Destrieux_lh_displacement    1.843224
Name: SHARE1080, dtype: float64